<a href="https://colab.research.google.com/github/lankston-consulting/rpms-cutter/blob/main/rpms_cutout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are some libraries that will need to be downloaded if the next cell throws some ModuleNotFound errors. Delete the # of the line who is missing and run the cell.

In [19]:
%%capture
!pip install rasterio
!pip install rioxarray
!pip install ipyleaflet

# **Library Imports and Back End Setup**
Nothing needs to be changed here, this cell just imports all the libraries our code needs to run, and gets Google Drive mounted to this notebook.

You **will** need to approve this notebook to having access to your local Google Drive, it will be a little pop up when this cell gets run!

In [22]:
import os
import datetime
import xarray as xr
import pandas as pd
import rasterio
import rioxarray
import dask
import tempfile
from ipyleaflet import *
from google.cloud import storage
from dask.distributed import Client, LocalCluster, Lock, as_completed, fire_and_forget
# from dotenv import load_dotenv
from google.colab import drive
# !gcloud auth login

drive.mount('/content/gdrive')
cluster = LocalCluster(n_workers=8, processes=True)
client = Client(cluster)
path_template = "gs://fuelcast-data/rpms/"
# load_dotenv()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43215 instead
  f"Port {expected} is already in use.\n"


# **Inputs** 

start_year: The beginning year of the range you would like.

end_year: The ending year of the range.

raster_file: The name, and optionally folder, of the file stored in gcp.

In [46]:
start_year = 2021
end_year = 2022
file_name = "degradation_bpslut4_bpslut4_wgs84.tif"


# **Optional geometry drawing tool**

In [57]:
poly_coords = []
zone_map = Map(center=(38, -97),
                zoom=6,
                basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik))
draw_control = DrawControl()
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.2
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}
def handle_draw(self, action, geo_json):
    print(poly_coords)
    poly_coords = draw_control.last_draw['geometry']['coordinates'][0]
    print(poly_coords)

draw_control.on_draw(handle_draw)

zone_map.add_control(draw_control)

zone_map



Map(center=[38, -97], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

UnboundLocalError: ignored

# **RPMS Extractor**
You don't need to change anything in here, just let the cell run.

In [50]:
if poly_coords is not "":
  print("in here")
  bounds = poly_coords

with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS="/content/gdrive/My Drive/fuelcast-storage-credentials.json"):
  zone_ds = rasterio.open("/content/" + file_name, chunks=(1024, 1024))
  bounds = zone_ds.bounds

  profile = zone_ds.profile
  profile.update(blockxsize=1024, blockysize=1024, tiled=True)

  for y in range(start_year, end_year):
      dx = rasterio.open(path_template + str(y) + "/rpms_" + str(y) + ".tif", chunks=(1, 1024, 1024), lock=False)
      op = f"rpms_{str(y)}_mean.tif"
      with rasterio.open('/content/gdrive/My Drive/rpms_'+str(y)+'.tif', 'w', **profile) as dst:
          win = dx.window(bottom=bounds.bottom, right=bounds.right, top=bounds.top, left=bounds.left)
          dat = dx.read(window=win)
          dst.write(dat)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: ignored